In [1]:
import mlflow
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import logging


logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [ ]:
mlflow.set_tracking_uri("sqlite:///backend.db")
mlflow.set_experiment("Experimento_3")

with mlflow.start_run(run_name = "example_1"):

    X,y = load_iris(return_X_y= True)
    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))
    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

### Probaremos que hacen las funciones que hay en el repositorio de Mlops

In [1]:
import pandas as pd
import numpy as np
import os
import json

In [2]:
def read_data(file_name: str, path="D:/Repositorio/Proyecto-titanic/Data") -> pd.DataFrame:
    return pd.read_csv(os.path.join(path, file_name))

In [3]:
data = read_data("titanic.csv")
data.head()

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500


In [4]:
def data_transform(data:pd.DataFrame):
    X = data[[ "Pclass", "Age", "Siblings/Spouses Aboard", "Parents/Children Aboard", "Fare", "Sex"]]
    y = data["Survived"]
    return X, y

In [21]:
X_titanic, y_titanic =data_transform(data)

In [6]:
def one_hot_encoding2(data, columns):
    encoded_data = pd.get_dummies(data, columns=columns, drop_first=True)
    return encoded_data.astype(int)


In [7]:
one_hot_encoding2(X, ["Sex"])

,Pclass,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare,Sex_male
0,3,22,1,0,7,1
1,1,38,1,0,71,0
2,3,26,0,0,7,0
3,1,35,1,0,53,0
4,3,35,0,0,8,1
...,...,...,...,...,...,...
882,2,27,0,0,13,1
883,1,19,0,0,30,0
884,3,7,1,2,23,0
885,1,26,0,0,30,1


In [8]:
def read_idx2label(json_path: str) -> pd.Series:
    """This function read the json file and return a dictionary
    Args:
      json_path (str): path to the json file
     Returns:
      idx2label (dict): dictionary with the mapping"""
    with open(json_path) as f:
        idx2label = json.load(f)
    return idx2label

In [11]:
def decode_labels_into_idx(labels: pd.Series, idx2label: dict) -> pd.Series:
    """This function decode the labels into idx
    Args:
      labels (pd.Series): series with the labels
      idx2label (dict): dictionary with the mapping
     Returns:
      labels (pd.Series): series with the labels decoded
    """
    return labels.map(idx2label)

In [28]:
tikets = pd.read_csv("tickets_inputs_eng_1.csv")
data_json= read_idx2label(json_path="topic_mapping_1.json")
label_tikets = {value: key for key, value in data_json.items()}
X, y = data_transform(tikets)
y = decode_labels_into_idx(labels=y, idx2label=label_tikets)
p

KeyError: "None of [Index(['Pclass', 'Age', 'Siblings/Spouses Aboard', 'Parents/Children Aboard',\n       'Fare', 'Sex'],\n      dtype='object')] are in the [columns]"

In [23]:
data_json

{'0': 'Bank Account Services',
 '1': 'Credit Report or Prepaid Card',
 '2': 'Mortgage/Loan'}